In [222]:
import re
import requests
import praw
import pickle
import itertools
from bs4 import BeautifulSoup
from time import sleep

In [183]:
s = "Hello's"
print(s.title())

Hello'S


In [203]:
s = 'Fox'


['FOX', 'FOx', 'FoX', 'Fox', 'fOX', 'fOx', 'foX', 'fox']

In [223]:
lis = ["no", "way", "lol"]
s = list(map(' '.join, itertools.product(*((c[0].upper() + c[1:], c) for c in lis))))
print(s)

['No Way Lol', 'No Way lol', 'No way Lol', 'No way lol', 'no Way Lol', 'no Way lol', 'no way Lol', 'no way lol']


In [224]:
m = re.compile(r"\[\[[^\]]*\]\]")

def supers(text):
    response = " "
    split = text.split()
    for i in range(len(split)):
        if i != 0:
            response += " "
        response += "^" + split[i]
    return response

def findItem(name):
    name = name.replace("'","%27").split()
    nameLen = len(name)
    for i in range(len(name)):
        test = itemLookup(name[:nameLen-i])
        if test != "":
            return test
            
    return ""
def itemLookup(name):
    # Try a URL
    possibleURLs = list(map('_'.join, itertools.product(*((c[0].upper() + c[1:].lower(), c.lower()) for c in name))))
    for possibleURL in possibleURLs:
        print(possibleURL)
        r = requests.get("http://www.pathofexile.gamepedia.com/" + possibleURL)
        if r.status_code != 404:
            break
    if r.status_code == 404:
        return ""
    
    soup = BeautifulSoup(r.text, "html.parser")
    itemspan = soup.find("span", { "class" : "infobox-page-container"})
    
    # Break if an invalid page has been reached
    if not itemspan:
        return ""
    
    itemspan = itemspan.find("span", { "class" : "item-box" })
    
    # Get item title
    itemTitleRaw = itemspan.find("span", { "class" : "header"}).children
    itemTitle = ""
    for item in itemTitleRaw:
        if item.string:
            itemTitle += item.string + " "
    itemTitle = itemTitle[:-1]
            
    # Get item stats
    itemStats = itemspan.find("span", { "class" : "item-stats"}).find("span")
    
    # Get item mods
    itemMods = itemspan.findAll("span", { "class" : "-mod"})
            
    # Get image URL
    itemURL = itemspan.findAll("a", { "class" : "image"})[0].img["src"]
    
    
    # Print
    response = ""
    response += "[**" + itemTitle + "**](" + itemURL + ")"
    response += "[[Wiki]](http://www.pathofexile.gamepedia.com/" + possibleURL + ")\n\n"
    
    # Print Stats

    for item in itemStats.children:
        try:
            temp = ""
            for child in item.children:
                if child.string:
                    temp += supers(child.string)
            response += temp
            if temp == "":
                response += " ^| "
        except:
            response += supers(item)
    response += "\n\n"
    
    #Print Mods
    for i in range(len(itemMods)):
        for item in itemMods[i].children:
            if item.string:
                if len(itemMods) == 2 and i == 0:
                    response += "*" + item.string + "*"
                else:
                    response += item.string
            else:
                response += "\n\n"
        response += "\n\n"

    return response

In [39]:
findItem("sorcerer boots bob")

In [100]:
r = praw.Reddit('bot1')

In [229]:
def respond(lim, rate):
    with open('ids.pickle', 'rb') as handle:
        ids = pickle.load(handle)
    while True:
        subreddit = r.subreddit("test")
        for submission in subreddit.new(limit=lim):
            comment_queue = submission.comments[:]
            while comment_queue:
                com = comment_queue.pop(0)
                if "[[" in com.body and "]]" in com.body and com.id not in ids:
                    ids.append(com.id)
                    print("Found Comment:" + com.id)
                    reply = ""
                    for item in m.findall(com.body)[:10]:
                        temp = findItem(item[2:-2])
                        reply += temp
                        if temp != "":
                            reply += "\n\n---------\n\n"
                    if reply != "":
                        reply += "I am a bot. Reply to me with up to 7 [[item names]]."
                        com.reply(reply)
                    else:
                        print("False Reply ^")
                comment_queue.extend(com.replies)
        with open('ids.pickle', 'wb') as handle:
            pickle.dump(ids, handle, protocol=pickle.HIGHEST_PROTOCOL)
        sleep(rate)

In [230]:
respond(50, 30)

Found Comment:dbjcylw
Essence_Of_Horror
Essence_Of_horror
Essence_of_Horror
Found Comment:dbjcz0x
Wiki
wiki
Item_Names
Item_names
item_Names
item_names
Item
False Reply ^


RequestException: error with request ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))